In [1]:
from gensim.models.doc2vec import Doc2Vec
import json
from PyDictionary import PyDictionary
from tqdm import tqdm
import numpy as np
from textblob import TextBlob
import inflect
p = inflect.engine()
dictionary=PyDictionary()

In [2]:
model1 = Doc2Vec.load('./doc2vec_dm1_5/doc2vec_model_dm1_add_5')
model2 = Doc2Vec.load('./doc2vec_dm0_5/doc2vec_model_dm0_add_5')

In [11]:
class find_model_sim:
    def __init__(self,model,vaild_words):
        self.model = model
        self.wv = model.wv
        self.vaild_words = vaild_words
        self.word_dict = self.wv.key_to_index
        vaild_index = [self.wv.key_to_index[x] for x in vaild_words]
        self.vaild_vector = self.wv.vectors[vaild_index]
        self.norms = np.linalg.norm(self.vaild_vector, axis=1)

    def find_most_sim_word(self,words,topn=None,a=0):
        results = []
        for word in words:
            if word not in self.wv:
                continue
            dists = np.dot(self.vaild_vector, self.wv[word]) / self.norms / np.linalg.norm(self.wv[word])
            sorted_dist = np.argsort(-dists)
            scores = dists[sorted_dist]
            word_sort = [(self.vaild_words[x],score) for x,score in zip(sorted_dist,scores)]
            word_sort = [x for x in word_sort if x[1] > a]
            if 'else' in [x[0] for x in word_sort]:
                print(word)
            if topn:
                results.append({'word':word,'sim_word':word_sort[:topn]})
            else:
                results.append({'word':word,'sim_word':word_sort})
        return results

In [4]:
reivew_vaild_words_1000 = []
with open('../new_1000.txt','r') as f:
    for i in f:
        reivew_vaild_words_1000.append(i.strip('\n'))

In [5]:
reivew_vaild_words_500 = []
with open('../new_500_1000.txt','r') as f:
    for i in f:
        reivew_vaild_words_500.append(i.strip('\n'))

In [6]:
game_vaild_words_50 = []
with open('../new_game_50.txt','r') as f:
    for i in f:
        game_vaild_words_50.append(i.strip('\n'))

In [7]:
total_word = reivew_vaild_words_1000 + reivew_vaild_words_500 + game_vaild_words_50

In [15]:
model_1_find = find_model_sim(model1,total_word)
model_2_find = find_model_sim(model2,total_word)

In [22]:
be_retrive_vaild_words = reivew_vaild_words_1000 + game_vaild_words_50 + reivew_vaild_words_500

In [25]:
model_1_find = find_model_sim(model1,be_retrive_vaild_words)
model_2_find = find_model_sim(model2,be_retrive_vaild_words)

In [26]:
items = []
already_deploy_word = []
with open('./core_word_rebuild2.json', 'r') as f:
    for i in f:
        item = json.loads(i)
        title = item['title']
        key_words = item['key_word']
        already_deploy_word += key_words
        key_words = [x for x in key_words if x in mother_vaild_words]
        items.append({'title':title,'old_keys':item['key_word'],'new_ori_key':key_words})

In [27]:
items = []
already_deploy_word = []
with open('./core_word_rebuild2.json', 'r') as f:
    for i in f:
        item = json.loads(i)
        title = item['title']
        key_words = item['key_word']
        already_deploy_word += key_words
        key_words = [x for x in key_words if x in mother_vaild_words]
        items.append({'title':title,'old_keys':item['key_word'],'new_ori_key':key_words})

        
deploy = {}
for i in tqdm(items):
    new_key_words = []
    title = i['title']
    new_ori_keys = i['new_ori_key']
    add_keys_words1 = model_1_find.find_most_sim_word(new_ori_keys, None, 0.75)
    add_keys_words2 = model_2_find.find_most_sim_word(new_ori_keys, None, 0.8)
    add_keys_words = add_keys_words1 + add_keys_words2
    temp_lists = []
    for word_group in add_keys_words:
        temp_lists += [x for x in word_group['sim_word'] if (x[0] not in already_deploy_word or x[0] in i['old_keys'])]
    temp_lists = sorted(temp_lists,key=lambda x:x[1],reverse=True)

    for word in temp_lists:
        if word[0] in deploy.keys():
            if word[1] >  deploy[word[0]][1]:
                deploy[word[0]] = (title,word[1])
        else:
            deploy[word[0]] = (title,word[1])

100%|██████████| 20/20 [01:04<00:00,  3.25s/it]
